In [271]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery
import time
%env GOOGLE_APPLICATION_CREDENTIALS=../secrets/bigquery-service-account.json 
client = bigquery.Client()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

env: GOOGLE_APPLICATION_CREDENTIALS=../secrets/bigquery-service-account.json


In [272]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [273]:
%%bigquery metadata --project masterarbeit-245718 --verbose 
SELECT * FROM `bigquery-public-data`.crypto_ethereum.INFORMATION_SCHEMA.COLUMNS where table_name = "traces"

Executing query with job ID: 0aef765d-2869-4895-80d2-6864d35a41b7
Query executing: 2.64s
Query complete after 5.22s


In [274]:
size_sample_data = 100

### Generate sample data for "call_type"

In [275]:
# %%bigquery call_types_sql_result --project masterarbeit-245718 --verbose 
# select DISTINCT call_type from `bigquery-public-data.crypto_ethereum.traces`
# where DATE(block_timestamp) >= '2019-07-6' AND DATE(block_timestamp) <= '2019-7-7'

In [276]:
call_types = ['call', 'delegatecall', 'staticcall', 'callcode', None]
prob_call_types = [0.7, 0.05, 0.025, 0.025, 0.2]

In [277]:
call_type_sample = np.random.choice(call_types, size_sample_data, p=prob_call_types)

### Generate sample data for "status"

In [278]:
# %%bigquery status_values --project masterarbeit-245718 --verbose 
# select DISTINCT status from `bigquery-public-data.crypto_ethereum.traces`
# where DATE(block_timestamp) >= '2019-07-6' AND DATE(block_timestamp) <= '2019-7-7'

In [279]:
status_values = [0, 1]
probs_status_values = [0.05, 0.95]

In [280]:
probs_status_values = [0.05, 0.95]

In [281]:
status_sample = np.random.choice(status_values, size_sample_data, p=probs_status_values)

### Generate sample accounts 

In [282]:
exchanges = ["exchange_{}".format(i) for i in range(1,6)]
exchanges[0:5]

['exchange_1', 'exchange_2', 'exchange_3', 'exchange_4', 'exchange_5']

In [283]:
speculators = ["speculator_{}".format(i) for i in range(1,101)]
speculators[0:5]

['speculator_1',
 'speculator_2',
 'speculator_3',
 'speculator_4',
 'speculator_5']

### Generate sample transactions (speculators to exchanges)

In [284]:
from_spec_addresses = [np.random.choice(speculators) for i in range(int(size_sample_data/2))]
to_ex_addresses = [np.random.choice(exchanges) for i in range(int(size_sample_data/2))]
values_spec_to_ex = np.random.randint(1, 20, int(size_sample_data/2))

In [285]:
txdata1 = pd.DataFrame(zip(from_spec_addresses, to_ex_addresses, values_spec_to_ex), columns=["from_address", "to_address", "value"])

### Generate sample transactions (exchanges to speculators)

In [286]:
to_spec_addresses = [np.random.choice(speculators) for i in range(int(size_sample_data/2))]
from_ex_addresses = [np.random.choice(exchanges) for i in range(int(size_sample_data/2))]
values_ex_to_spec = np.random.randint(1, 5, int(size_sample_data/2))

In [287]:
txdata2 = pd.DataFrame(zip(from_ex_addresses, to_spec_addresses, values_ex_to_spec), columns=["from_address", "to_address", "value"])

Anmerkung: Die speculators schicken mehr Geld zu den Börsen als umgekehrt.

In [288]:
txdata = txdata1.append(txdata2).reset_index(drop=True)

### Generate sample "block_timestamps"

In [289]:
import datetime as datetime

base = datetime.datetime.utcnow()
block_timestamps = [base - datetime.timedelta(seconds=x) for x in range(0, 12*size_sample_data, 12)]

### Merge data to sample "traces" table

In [290]:
traces_sampleData = txdata.copy()

In [291]:
traces_sampleData["status"] = status_sample 

In [292]:
traces_sampleData["call_type"] = call_type_sample 

In [293]:
traces_sampleData["block_timestamp"] = block_timestamps 

In [294]:
traces_sampleData.head()

,from_address,to_address,value,status,call_type,block_timestamp
0,speculator_84,exchange_3,4,1,call,2020-01-22 13:55:25.766665
1,speculator_74,exchange_5,13,1,call,2020-01-22 13:55:13.766665
2,speculator_15,exchange_2,19,1,call,2020-01-22 13:55:01.766665
3,speculator_41,exchange_4,11,1,call,2020-01-22 13:54:49.766665
4,speculator_3,exchange_1,6,1,call,2020-01-22 13:54:37.766665


### Upload "traces" table to bigquery

In [295]:
traces_sampleData.to_gbq('ethereum_us.traces_sampleData', if_exists="replace")

1it [00:02,  3.00s/it]


### Test SQL command for retrieving "weiReceived", "weiSent"

In [296]:
%%bigquery res1 --project masterarbeit-245718 --verbose 

  with weiReceivedView as (
      
    -- debits
    select to_address, sum(ifnull(value, 0)) as weiReceived
    from ethereum_us.traces_sampleData
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
    group by to_address
      
), weiSentView as (
    -- credits
    select from_address, sum(ifnull(value, 0)) as weiSent
    from ethereum_us.traces_sampleData
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
    group by from_address
) 
select 
    CASE 
      when to_address is not null then to_address
      when from_address is not null then from_address
    end as address, 
    ifnull(weiReceived,0) as weiReceived, 
    ifnull(weiSent,0) as weiSent
from weiReceivedView full outer join weiSentView on from_address = to_address

Executing query with job ID: a5942866-b9ad-4a35-94c8-d6a28849e071
Query executing: 0.65s
Query complete after 3.14s


In [297]:
res2 = res1.copy()
res2["balance"] = res1.weiReceived - res1.weiSent
res2 = res2.set_index("address")
res2 = res2.sort_values(by="address", ascending=False)
res2 = res2.astype("float")
balance_result_sql = res2

In [298]:
%%bigquery traces_sampleData --project masterarbeit-245718 --verbose 
select * from `ethereum_us.traces_sampleData`

Executing query with job ID: d650859c-57e8-44f6-99fc-a6f42116051c
Query executing: 0.57s
Query complete after 1.34s


In [299]:
data1 = [row for (index, row) in traces_sampleData.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or call_type == None) and row.status == 1]
data1 = pd.DataFrame(data1)

In [300]:
data2 = pd.DataFrame(index=set(data1["from_address"].unique()) | set(data1["to_address"].unique()))
data2["weiReceived"] = data1.groupby('to_address').sum().value
data2["weiSent"] = data1.groupby('from_address').sum().value
data2["weiSent"] = data2["weiSent"].fillna(0.)
data2["weiReceived"] = data2["weiReceived"].fillna(0.)
data2["balance"] = data2["weiReceived"] - data2["weiSent"]
data2.index = data2.index.rename("address")
data2 = data2.sort_values(by="address", ascending=False)
balance_result_py = data2

In [301]:
pd.testing.assert_frame_equal(balance_result_py, balance_result_sql)
print("weiSent, weiReceived Test succeeded!!")

weiSent, weiReceived Test succeeded!!


### Test SQL command for retrieving "txSent" and "txReceived"

In [302]:
%%bigquery res2 --project masterarbeit-245718 --verbose 
with txSent as (
  SELECT from_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
  where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
  group by from_address
), txReceived as (
  SELECT to_address, count(*) as numberOfTranscationsReceived FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
  where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
  group by to_address
) SELECT 
CASE  
  WHEN to_address IS NOT NULL THEN to_address
  WHEN from_address IS NOT NULL THEN from_address
END AS address,
IFNULL(numberOfTranscationsReceived, 0) as numberOfTranscationsReceived, 
IFNULL(numberOfTranscationsSent, 0) as numberOfTranscationsSent
from txReceived FULL OUTER JOIN txSent on to_address = from_address

Executing query with job ID: bc265081-669f-4d07-b64b-67d5bf4db265
Query executing: 0.63s
Query complete after 1.69s


In [303]:
res3 = res2.copy()
res3 = res3.sort_values(by="address", ascending=False)
tx_sent_received_result_sql = res3.set_index("address", drop=True)

In [304]:
data3 = pd.DataFrame(index=set(data1["from_address"].unique()) | set(data1["to_address"].unique()))
data3["numberOfTranscationsReceived"] = data1.groupby('to_address').count().value
data3["numberOfTranscationsSent"] = data1.groupby('from_address').count().value
data3 = data3.fillna(0)
data3 = data3.astype("int")
data3.index = data3.index.rename("address")
data3 = data3.sort_values(by="address", ascending=False)
tx_sent_received_result_py = data3

In [305]:
pd.testing.assert_frame_equal(tx_sent_received_result_py, tx_sent_received_result_sql)
print("txSent, txReceived Test succeeded !!")

txSent, txReceived Test succeeded !!


In [306]:
tx_sent_received_result_py

,numberOfTranscationsReceived,numberOfTranscationsSent
address,,
speculator_99,0,1
speculator_96,0,1
speculator_93,2,2
speculator_91,0,1
speculator_90,2,0
speculator_9,0,1
speculator_89,0,1
speculator_88,1,0
speculator_85,1,0


In [307]:
tx_sent_received_result_sql

,numberOfTranscationsReceived,numberOfTranscationsSent
address,,
speculator_99,0,1
speculator_96,0,1
speculator_93,2,2
speculator_91,0,1
speculator_90,2,0
speculator_9,0,1
speculator_89,0,1
speculator_88,1,0
speculator_85,1,0


### Test avg time between tx

In [308]:
%%bigquery res4 --project masterarbeit-245718 --verbose 
with receivedTx as (
  SELECT to_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
  where to_address is not null 
    and status = 1 
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
  group by to_address),
timeStampDiffs as (
  SELECT to_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
  FROM `masterarbeit-245718.ethereum_us.traces_sampleData`
  where to_address is not null 
    and status = 1 
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
  group by to_address
) select to_address as address, 
CASE 
  when (numberOfTranscationsSent - 1)  > 0 then timestampDiff / (numberOfTranscationsSent - 1) 
  else 0
end as avgTimeDiffBetweenReceivedTransactions
   from receivedTx inner join  timeStampDiffs using(to_address)

Executing query with job ID: b5eac19b-4c05-47d1-8537-868ab93e3a82
Query executing: 0.49s
Query complete after 1.91s


In [309]:
res6 = res4.set_index("address", drop=True)
res6 = res6.sort_values(by="address")
avg_time_diff_receivedtx_result_sql = res6
avg_time_diff_receivedtx_result_sql

,avgTimeDiffBetweenReceivedTransactions
address,
exchange_1,86.000000
exchange_2,46.909091
exchange_3,68.571429
exchange_4,61.333333
exchange_5,62.666667
speculator_100,0.000000
speculator_12,0.000000
speculator_13,0.000000
speculator_17,0.000000


In [310]:
%%bigquery res5 --project masterarbeit-245718 --verbose 
with sentTx as (
  SELECT from_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
  where to_address is not null 
    and status = 1 
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
  group by from_address),
timeStampDiffs as (
  SELECT from_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
  FROM `masterarbeit-245718.ethereum_us.traces_sampleData`
  where to_address is not null 
    and status = 1 
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
  group by from_address
) select from_address as address, 
CASE 
  when (numberOfTranscationsSent - 1)  > 0 then timestampDiff / (numberOfTranscationsSent - 1) 
  else 0
end as avgTimeDiffBetweenSentTransactions
   from sentTx inner join  timeStampDiffs using(from_address)

Executing query with job ID: 5a094e6c-998e-4a9d-b6e5-43b41d6fb6b9
Query executing: 0.50s
Query complete after 1.81s


In [311]:
res6 = res5.set_index("address", drop=True)
res6 = res6.sort_values(by="address")
avg_time_diff_senttx_result_sql = res6
avg_time_diff_senttx_result_sql

,avgTimeDiffBetweenSentTransactions
address,
exchange_1,55.500000
exchange_2,48.000000
exchange_3,86.400000
exchange_4,77.142857
exchange_5,78.000000
speculator_1,0.000000
speculator_100,0.000000
speculator_12,0.000000
speculator_13,0.000000


In [312]:
res7 = [row for (index, row) in traces_sampleData.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or call_type == None) and row.status == 1]
res7 = pd.DataFrame(res7)
res8 = res7.groupby("from_address").max().block_timestamp
res9 = res7.groupby("from_address").min().block_timestamp
res10 = res8 - res9
res10 = res10.rename("seconds_diff")
res10 = tx_sent_received_result_py.join(res10, how="right").drop("numberOfTranscationsReceived", axis=1)
res10 = res10.fillna(0.)
res10["avgTimeDiffBetweenSentTransactions"] = res10["seconds_diff"] / (res10["numberOfTranscationsSent"] - 1)
res10["avgTimeDiffBetweenSentTransactions"] = res10["avgTimeDiffBetweenSentTransactions"].fillna(datetime.timedelta(0))
res10.index =res10.index.rename("address")
res10 = res10.sort_values(by="address")
res10 = res10.drop(["numberOfTranscationsSent", "seconds_diff"],axis=1)
res10["avgTimeDiffBetweenSentTransactions"] = [ts.total_seconds() for ts in res10["avgTimeDiffBetweenSentTransactions"]]
avg_time_diff_senttx_result_py = res10
avg_time_diff_senttx_result_py

,avgTimeDiffBetweenSentTransactions
address,
exchange_1,55.500000
exchange_2,48.000000
exchange_3,86.400000
exchange_4,77.142857
exchange_5,78.000000
speculator_1,0.000000
speculator_100,0.000000
speculator_12,0.000000
speculator_13,0.000000


In [313]:
pd.testing.assert_frame_equal(avg_time_diff_senttx_result_sql, avg_time_diff_senttx_result_py)
print("avgTimeDiffBetweenSentTransactions Test succeeded !!")

avgTimeDiffBetweenSentTransactions Test succeeded !!


In [314]:
features = balance_result_sql.join(tx_sent_received_result_sql,how="left")
features = features.join(avg_time_diff_senttx_result_sql)
features = features.join(avg_time_diff_receivedtx_result_sql)
features = features.sort_values(by="balance", ascending=False)
features = features.fillna(0.0)
import sys
# addresses that have sent/received only one transaction get the avg time max * 2
features["avgTimeDiffBetweenSentTransactions"] = features["avgTimeDiffBetweenSentTransactions"].replace(to_replace=0.0, value=2 * max(features["avgTimeDiffBetweenSentTransactions"]))
features["avgTimeDiffBetweenReceivedTransactions"] = features["avgTimeDiffBetweenReceivedTransactions"].replace(to_replace=0.0, value=2 * max(features["avgTimeDiffBetweenReceivedTransactions"]))

In [315]:
features = features.reset_index()

In [317]:
features.head()

,address,weiReceived,weiSent,balance,numberOfTranscationsReceived,numberOfTranscationsSent,avgTimeDiffBetweenSentTransactions,avgTimeDiffBetweenReceivedTransactions
0,exchange_2,132.0,37.0,95.0,12,13,48.000000,46.909091
1,exchange_4,85.0,22.0,63.0,10,8,77.142857,61.333333
2,exchange_5,74.0,16.0,58.0,10,7,78.000000,62.666667
3,exchange_3,58.0,14.0,44.0,8,6,86.400000,68.571429
4,exchange_1,49.0,28.0,21.0,7,9,55.500000,86.000000


In [318]:
features.to_gbq('ethereum_us.features_sampleData', if_exists="replace")

1it [00:04,  4.32s/it]


# Deprecated